In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
from glob import glob
from sklearn.preprocessing import StandardScaler
import joblib

# Hàm trích xuất
def extract_audio_features(audio_path):
    y, sr = librosa.load(audio_path, sr=16000)
    y = librosa.util.fix_length(y, size=5 * sr)


    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    pitch, mag = librosa.piptrack(y=y, sr=sr)
    pitches = pitch[mag > np.median(mag)]
    pitch_mean = np.mean(pitches) if len(pitches) > 0 else 0

    zcr = np.mean(librosa.feature.zero_crossing_rate(y))
    energy = np.mean(y ** 2)
    centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))

    return {
        "mfcc": np.mean(mfcc, axis=1).tolist(),
        "pitch": float(pitch_mean),
        "energy": float(energy),
        "zcr": float(zcr),
        "centroid": float(centroid)
    }

# Trích xuất toàn bộ file trong folder
def save_features_to_csv(folder_path, output_csv):
    data = []
    for file in glob(os.path.join(folder_path, "*.wav")):
        features = extract_audio_features(file)
        row = {
            "filename": os.path.basename(file),
            "pitch": features["pitch"],
            "energy": features["energy"],
            "zcr": features["zcr"],
            "centroid": features["centroid"]
        }
        # Thêm MFCCs vào từng cột riêng
        for i, val in enumerate(features["mfcc"]):
            row[f"mfcc_{i+1}"] = val
        data.append(row)
    
    df = pd.DataFrame(data)
    df.to_csv(output_csv, index=False)
    print(f"Đã lưu đặc trưng vào {output_csv}")
    
    #  CHUẨN HÓA nếu có dữ liệu
    if len(df) > 0:

        # Bỏ cột 'filename' nếu tồn tại
        feature_cols = [col for col in df.columns if col != "filename"]

        scaler = StandardScaler()
        df_scaled = df.copy()
        df_scaled[feature_cols] = scaler.fit_transform(df[feature_cols])

        df_scaled.to_csv("voice_features_normalized.csv", index=False)
        joblib.dump(scaler, "scaler.pkl")

        print(" Đã chuẩn hóa và lưu voice_features_normalized.csv")
        print(" Đã lưu scaler.pkl")
    else:
        print("⚠️ Không có dữ liệu để chuẩn hóa.")
    
    
# Gọi hàm
save_features_to_csv("D:\\Project\\Python\\Nhan_dien_giong_noi_csdldpt\\data", "voice_features.csv")
print(" Đã hoàn thành trích xuất đặc trưng âm thanh")


Đã lưu đặc trưng vào voice_features.csv
✅ Đã chuẩn hóa và lưu voice_features_normalized.csv
✅ Đã lưu scaler.pkl
 Đã hoàn thành trích xuất đặc trưng âm thanh


In [ ]:
import pandas as pd
import mysql.connector
import json

# Đọc file CSV
df = pd.read_csv("voice_features.csv")

# Kết nối MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123456",
    database="voice_db"
)
cursor = conn.cursor()

# Vòng lặp để insert từng dòng
for _, row in df.iterrows():
    mfcc = [row[f"mfcc_{i+1}"] for i in range(13)]  # Tạo list MFCC

    sql = """
    INSERT INTO voices (name, mfcc, pitch, energy, zcr, centroid)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    cursor.execute(sql, (
        row["filename"],
        json.dumps(mfcc),            # chuyển MFCC thành chuỗi JSON
        row["pitch"],
        row["energy"],
        row["zcr"],
        row["centroid"]
    ))

conn.commit()
conn.close()
print(" Đã nhập dữ liệu từ CSV vào MySQL thành công.")


